# OpenAI Agents Orchestration Demo

Use this notebook to explore how `OpenAIAgentOrchestrator` turns the reconciliation engine into an OpenAI Agents workflow. Update the payload cells with your own GL/subledger data once you are ready to run a live reconciliation.

## Prerequisites

- Install dependencies from the repo root: `pip install uv` (if needed) then `uv sync`
- Copy `.env.sample` to `.env` and set `OPENAI_API_KEY` (required) plus `GEMINI_API_KEY` or `GOOGLE_API_KEY` if you want Gemini commentary
- Start the notebook with the project virtual environment activated so imports resolve correctly

In [ ]:
# Optional: load environment variables from .env when running locally.
from pathlib import Path

env_path = Path(".env")
if env_path.exists():
    from dotenv import load_dotenv

    load_dotenv(env_path)
else:
    print("No .env file found; make sure OPENAI_API_KEY is set in your shell.")

In [ ]:
from recon_agent import (
    AgentConfig,
    GeminiConfig,
    GeminiInsightGenerator,
    GeminiLLM,
    OpenAIAgentOrchestrator,
    ReconciliationAgent,
)

reconciliation = ReconciliationAgent(
    config=AgentConfig(materiality_threshold=10),
    insight_generator=GeminiInsightGenerator(GeminiLLM(GeminiConfig())),
)

orchestrator = OpenAIAgentOrchestrator(reconciliation)

## Prepare sample data

Update the data below with your GL and subledger balances. The defaults mirror the simplified example shown in the docs.

In [ ]:
user_prompt = "Reconcile inventory control account 1000 for October."

tool_payload = {
    "gl_balances": [
        {"account": "1000", "period": "2024-10", "amount": 120000.0},
    ],
    "subledger_balances": [
        {"account": "1000", "period": "2024-10", "amount": 118500.0},
    ],
    "transactions": [
        {
            "account": "1000",
            "booked_at": "2024-10-15",
            "description": "Inventory adjustment",
            "amount": -1500.0,
        }
    ],
}


## Run the orchestrator

The supervisor role will call the reconciliation tool. The reviewer sees the resulting JSON and produces an executive summary. Running this cell requires a valid `OPENAI_API_KEY` and network access.

In [ ]:
reply = orchestrator.run(user_prompt, tool_payload=tool_payload)

print("Supervisor:\n", reply.message)
print("\nReviewer:\n", reply.messages_by_role.get("reviewer", "(no reviewer message)"))

## Inspect structured output

The raw reconciliation payload can feed downstream exports, dashboards, or audit logs.

In [ ]:
import json

print(json.dumps(reply.tool_output, indent=2, default=str))
